In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
# !pip install pymysql
# import pymysql

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [2]:
df_train_c = pd.read_csv('01_M02_DC_train_pre.csv').iloc[:, 1:]
df_ttf_dup = pd.read_csv('01_M02_DC_train_ttf_dup.csv')
df = pd.merge(df_train_c, df_ttf_dup.iloc[:,:2], how='left', on='time')
df = df.dropna()
df = df.reset_index(drop=True)
df[:5]

,time,Tool,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,...,ETCHPBNGASREADBACK,FIXTURETILTANGLE,ROTATIONSPEED,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION,TTF_FlowCool Pressure Dropped Below Limit
0,3283834,01M02,1,9253,10511673,67,1,-0.079428,-0.958814,-0.959509,...,-2.783162,2.027848,-0.013207,-0.122437,0.0,2.39537,2.871511,3.175941,4.037785,623234.0
1,3283838,01M02,1,9253,10511673,67,1,-0.079419,-0.958928,-0.959651,...,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.39537,2.871511,3.175941,4.037785,623230.0
2,3283842,01M02,1,9253,10511673,67,1,-0.079409,-0.959156,-0.960006,...,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.39537,2.871511,3.175941,4.037785,623226.0
3,3283846,01M02,1,9253,10511673,67,1,-0.079420,-0.958928,-0.959297,...,-2.780884,2.027848,-0.013207,-0.122437,0.0,2.39537,2.871511,3.175941,4.037785,623222.0
4,3283850,01M02,1,9253,10511673,67,1,-0.079430,-0.959269,-0.959438,...,-2.784389,2.027848,-0.013207,-0.122437,0.0,2.39537,2.871511,3.175941,4.037785,623218.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4621786 entries, 0 to 4621785
Data columns (total 25 columns):
 #   Column                                     Dtype  
---  ------                                     -----  
 0   time                                       int64  
 1   Tool                                       object 
 2   stage                                      int64  
 3   Lot                                        int64  
 4   runnum                                     int64  
 5   recipe                                     int64  
 6   recipe_step                                int64  
 7   IONGAUGEPRESSURE                           float64
 8   ETCHBEAMVOLTAGE                            float64
 9   ETCHBEAMCURRENT                            float64
 10  ETCHSUPPRESSORVOLTAGE                      float64
 11  ETCHSUPPRESSORCURRENT                      float64
 12  FLOWCOOLFLOWRATE                           float64
 13  FLOWCOOLPRESSURE                          

In [4]:
df2 = df[:]
temp = df2.iloc[:, 7:-1]
dfx = temp[temp.columns.difference(['FIXTURESHUTTERPOSITION'])]
dfy = df2.iloc[:, -1]
display(dfx[:5], dfy[:5])

,ACTUALROTATIONANGLE,ACTUALSTEPDURATION,ETCHAUX2SOURCETIMER,ETCHAUXSOURCETIMER,ETCHBEAMCURRENT,ETCHBEAMVOLTAGE,ETCHGASCHANNEL1READBACK,ETCHPBNGASREADBACK,ETCHSOURCEUSAGE,ETCHSUPPRESSORCURRENT,ETCHSUPPRESSORVOLTAGE,FIXTURETILTANGLE,FLOWCOOLFLOWRATE,FLOWCOOLPRESSURE,IONGAUGEPRESSURE,ROTATIONSPEED
0,-0.122437,4.037785,3.175941,2.871511,-0.959509,-0.958814,-1.751661,-2.783162,2.39537,-0.939043,-0.946194,2.027848,-2.842907,-2.332807,-0.079428,-0.013207
1,-0.122437,4.037785,3.175941,2.871511,-0.959651,-0.958928,-1.743905,-2.780884,2.39537,-0.939043,-0.946103,2.027848,-2.842907,-2.332807,-0.079419,-0.013207
2,-0.122437,4.037785,3.175941,2.871511,-0.960006,-0.959156,-1.743905,-2.780884,2.39537,-0.939043,-0.946057,2.027848,-2.841850,-2.332807,-0.079409,-0.013207
3,-0.122437,4.037785,3.175941,2.871511,-0.959297,-0.958928,-1.747158,-2.780884,2.39537,-0.939043,-0.946148,2.027848,-2.841850,-2.332807,-0.079420,-0.013207
4,-0.122437,4.037785,3.175941,2.871511,-0.959438,-0.959269,-1.747158,-2.784389,2.39537,-0.939043,-0.946057,2.027848,-2.841850,-2.332807,-0.079430,-0.013207


0    623234.0
1    623230.0
2    623226.0
3    623222.0
4    623218.0
Name: TTF_FlowCool Pressure Dropped Below Limit, dtype: float64

In [11]:
df[df['TTF_FlowCool Pressure Dropped Below Limit'] == 0]

,time,Tool,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,...,ETCHPBNGASREADBACK,FIXTURETILTANGLE,ROTATIONSPEED,ACTUALROTATIONANGLE,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION,TTF_FlowCool Pressure Dropped Below Limit
131029,3907068,01M02,1,9361,10548007,67,43,-0.022398,1.188755,0.789706,...,0.358499,0.022131,-0.013207,-0.122455,1.0,2.787074,3.309810,3.631607,0.803526,0.0
144061,3962388,01M02,1,9194,10551521,67,49,0.011108,1.187162,0.786655,...,0.352016,0.022131,-0.013207,-0.122455,1.0,2.802185,3.326749,3.649218,0.803526,0.0
146073,3971424,01M02,1,9194,10551521,67,46,-0.050470,-0.840118,-0.959509,...,0.326081,-0.646442,-0.013207,-0.122455,0.0,2.806268,3.331319,3.653969,0.803526,0.0
146554,3973348,01M02,1,9194,10551521,67,52,-0.079494,-0.959099,-0.959580,...,-2.787718,-0.646442,-0.013207,-0.122464,0.0,2.806501,3.331583,3.654243,0.803526,0.0
816393,8072052,01M02,1,9289,10810825,67,25,0.013171,0.711466,1.375094,...,0.357450,0.022131,-0.013207,-0.122455,1.0,0.523792,0.777750,0.999215,-0.304990,0.0
822293,8123030,01M02,1,9295,10815254,67,19,0.013694,0.950167,0.990015,...,0.362004,-0.646442,-0.013207,-0.122455,1.0,0.526125,0.780385,1.001955,-0.683190,0.0
822782,8124986,01M02,1,9295,10815254,67,27,-0.079422,-0.958246,-0.958942,...,-2.787718,-0.646442,-0.013207,-0.122464,0.0,0.526417,0.780716,1.002298,-0.657107,0.0
822894,8125434,01M02,1,9295,10815254,67,37,-0.075054,-0.959042,-0.959013,...,-2.787718,2.027848,-0.013207,-0.122455,0.0,0.526652,0.780983,1.002576,-0.304990,0.0
823316,8127122,01M02,1,9295,10815254,67,43,0.014220,0.710330,1.244443,...,0.357450,-0.646442,-0.013207,-0.122446,0.0,0.526848,0.781209,1.002811,-0.304990,0.0
2203089,18108796,01M02,88,3605,11472802,68,3,0.004714,1.546785,0.771974,...,0.360776,-0.200727,-0.013207,-0.122446,1.0,0.040941,0.243947,0.444260,-0.604941,0.0


In [10]:
dfx = temp[['ETCHAUXSOURCETIMER', 'ETCHAUX2SOURCETIMER', 'ETCHSOURCEUSAGE', 'ETCHBEAMCURRENT', 'ETCHBEAMVOLTAGE', 'ETCHSUPPRESSORCURRENT', 'ETCHSUPPRESSORVOLTAGE']]
dfy = df2.iloc[:, -1]

In [11]:
train_input, test_input, train_target, test_target = train_test_split(dfx, dfy, test_size=0.2, random_state=42)

In [16]:
### 특성공학
# 변환기 생성
# include_bias=False : y절편값은 생성하지 않음
poly = PolynomialFeatures(degree=3, include_bias=False)

### 특성 조합 패턴 찾아내기
# 훈련데이터만 넣을 수 있음 !
poly.fit(train_input)

### 찾은 조합을 이용해서 자동 변환 및 생성 시키기
# 훈련데이터 변환하기
train_poly = poly.transform(train_input)

# 테스트데이터 변환하기
test_poly = poly.transform(test_input)

In [12]:
### 다중회귀분석
# 모델생성
lr = LinearRegression()

# 훈련
lr.fit(train_poly, train_target)

# # 과적합 여부 확인
print("훈련 정확도 : ", lr.score(train_poly, train_target))
print("테스트 정확도 : ", lr.score(test_poly, test_target))

# 평균오차 확인
test_prod = lr.predict(test_poly)
print('MSE : ', mean_squared_error(test_prod, test_target))

훈련 정확도 :  0.24025200478426667
테스트 정확도 :  0.24007062853373984
MSE :  3933874115000.0967


In [12]:
tr_lis = []
te_lis = []
mse_lis = []
i = 3

for i in range(3,20,2):
    poly = PolynomialFeatures(degree=i, include_bias=False)

    ### 특성 조합 패턴 찾아내기
    # 훈련데이터만 넣을 수 있음 !
    poly.fit(train_input)

    ### 찾은 조합을 이용해서 자동 변환 및 생성 시키기
    # 훈련데이터 변환하기
    train_poly = poly.transform(train_input)

    # 테스트데이터 변환하기
    test_poly = poly.transform(test_input)
    
    ### 다중회귀분석
    # 모델생성
    lr = LinearRegression()

    # 훈련
    lr.fit(train_poly, train_target)

    # 평균오차 확인
    test_pred = lr.predict(test_poly)
    
    # 값저장
    tr = lr.score(train_poly, train_target)
    te = lr.score(test_poly, test_target)
    mse = mean_squared_error(test_pred, test_target)
    
    tr_lis.append(tr)
    te_lis.append(te)
    mse_lis.append(mse)
    
    if tr>=0.90:
        break

    # 과적합 여부 확인
    print(i, '-'*30)
    i += 2

3 ------------------------------


In [8]:
i, tr_lis, te_lis, mse_lis

(3, [0.9885331041496156], [-1.5385251229990102e+17], [2.0197296955583155e+29])

In [ ]:
823317